In [28]:
import numpy as np
import pandas as pd
import seaborn as sns
import nibabel as nib
import matplotlib.pyplot as plt
import nibabel.freesurfer.mghformat as mgh

from glob import glob
from os.path import join, exists
from nilearn import plotting, datasets, image

In [2]:
sns.set(style='whitegrid', context='talk')
plt.rcParams["font.family"] = "monospace"
plt.rcParams['font.monospace'] = 'Courier New'

In [3]:
ABCD_DIR = "/Volumes/projects_herting/LABDOCS/PROJECTS/ABCD/Data/release4.0"
PROJ_DIR = "/Volumes/projects_herting/LABDOCS/Personnel/Katie/deltaABCD_clustering/"
DATA_DIR = "data/"
FIGS_DIR = "figures/"
OUTP_DIR = "output/"
RESR_DIR = "resources/"

In [4]:
data_dictionary = pd.read_csv(join(ABCD_DIR, 'generate_dataset/data_element_names.csv'), 
                              header=0, 
                              index_col=0)

In [5]:
data = pd.read_csv(join(PROJ_DIR, DATA_DIR, "data.csv"), index_col=0, header=0)

In [6]:
smri_vars = pd.concat([data.filter(regex='smri.*change_score'), 
                                data.filter(regex='mrisdp.*change_score')], axis=1).dropna(how='all').columns
rsfmri_vars = data.filter(regex='rsfmri.*change_score').dropna(how='all').columns
rsi_vars = data.filter(regex='dmri_rsi.*change_score').dropna(how='all').columns
dti_vars = data.filter(regex='dmri_dti.*change_score').dropna(how='all').columns

In [7]:
smri_vars = [v.split('.')[0] for v in smri_vars]
rsfmri_vars = [v.split('.')[0] for v in rsfmri_vars]
rsi_vars = [v.split('.')[0] for v in rsi_vars]
dti_vars = [v.split('.')[0] for v in dti_vars]

In [8]:
imaging_vars = list(smri_vars) + list(rsfmri_vars) + list(rsi_vars) + list(dti_vars)

In [9]:
mapping = pd.DataFrame(columns=['modality', 
                                'abcd_structure',
                                'abcd_description', 
                                'atlas_description', 
                                'atlas', 
                                'atlas_value',
                                'atlas_fname'])

In [10]:
destrieux = datasets.fetch_atlas_surf_destrieux()
destrieux_vol = datasets.fetch_atlas_destrieux_2009()
desikan = datasets.fetch_neurovault_ids(image_ids=(23262, ))
#gordon
subcort = datasets.fetch_atlas_harvard_oxford('sub-maxprob-thr50-2mm')


Reading local neurovault data.
Already fetched 1 image
1 image found on local disk.


/Users/katherine.b/Library/Python/3.8/lib/python/site-packages/numpy/lib/npyio.py:2407: VisibleDeprecationWarning: Reading unicode strings without specifying the encoding argument is deprecated. Set the encoding, use None for the system default.
  output = genfromtxt(fname, **kwargs)


In [11]:
scs_nifti = subcort.maps

In [12]:
nib.save(scs_nifti, join(PROJ_DIR, RESR_DIR, 'harvox-subcortical-maxprob-thr50-2mm'))

In [60]:
for i in imaging_vars:
    if i in smri_vars:
        mapping.at[i, 'modality'] = 'smri'
    elif i in rsfmri_vars:
        mapping.at[i, 'modality'] = 'fmri' 
    elif i in rsi_vars:
        mapping.at[i, 'modality'] = 'rsi'
    elif i in dti_vars:
        mapping.at[i, 'modality'] = 'dti'
    mapping.at[i, 'abcd_description'] = data_dictionary.loc[i]['description']
    mapping.at[i, 'abcd_structure'] = data_dictionary.loc[i]['structure']
    if '_scs_' in i:
        mapping.at[i, 'atlas'] = 'Subcortical'
        mapping.at[i, 'atlas_fname'] = join(PROJ_DIR, RESR_DIR, 'harvox-subcortical-maxprob-thr50-2mm.nii')
    elif 'gordon' in i:
        mapping.at[i, 'atlas'] = 'Gordon'
        #mapping.at[i, 'atlas_fname'] = 
    elif '_cdk_' in i:
        mapping.at[i, 'atlas'] = 'Desikan'
        mapping.at[i, 'atlas_fname'] = desikan.images[0]
    elif 'mrisdp' in i:
        mapping.at[i, 'atlas'] = 'Destrieux'
        mapping.at[i, 'atlas_fname'] = destrieux_vol.maps
    elif '_cf12_' in i:
        # remove
        mapping.at[i, 'atlas'] = 'Fuzzy 12'
    elif 'fib' in i:
        mapping.at[i, 'atlas'] = 'Fiber Atlas'

In [14]:
l_and_r_destrieux = destrieux.labels * 2
for i in range(0, len(l_and_r_destrieux)):
    if i == 0:
        pass
    elif i < 42:
        mapping.at[f'mrisdp_{i}','atlas_description'] = l_and_r_destrieux[i]
        mapping.at[f'mrisdp_{i}','atlas_value'] = i
        mapping.at[f'mrisdp_{302 + i}','atlas_description'] = l_and_r_destrieux[i]
        mapping.at[f'mrisdp_{302 + i}','atlas_value'] = i
        mapping.at[f'mrisdp_{906 + i}','atlas_description'] = l_and_r_destrieux[i]
        mapping.at[f'mrisdp_{906 + i}','atlas_value'] = i
    elif i == 42:
        pass
    elif 42 < i < 76:
        mapping.at[f'mrisdp_{i - 1}','atlas_description'] = l_and_r_destrieux[i]
        mapping.at[f'mrisdp_{i - 1}','atlas_value'] = i
        mapping.at[f'mrisdp_{302 + i - 1}','atlas_description'] = l_and_r_destrieux[i]
        mapping.at[f'mrisdp_{302 + i - 1}','atlas_value'] = i
        mapping.at[f'mrisdp_{906 + i - 1}','atlas_description'] = l_and_r_destrieux[i]
        mapping.at[f'mrisdp_{906 + i - 1}','atlas_value'] = i
    elif i == 76:
        pass
    elif 76 < i < 116:
        mapping.at[f'mrisdp_{i - 2}','atlas_description'] = l_and_r_destrieux[i]
        mapping.at[f'mrisdp_{i - 2}','atlas_value'] = i
        mapping.at[f'mrisdp_{302 + i - 2}','atlas_description'] = l_and_r_destrieux[i]
        mapping.at[f'mrisdp_{302 + i - 2}','atlas_value'] = i
        mapping.at[f'mrisdp_{906 + i - 2}','atlas_description'] = l_and_r_destrieux[i]
        mapping.at[f'mrisdp_{906 + i - 2}','atlas_value'] = i
    elif i == 116:
        pass
    elif i > 116:
        mapping.at[f'mrisdp_{i - 3}','atlas_description'] = l_and_r_destrieux[i]
        mapping.at[f'mrisdp_{i - 3}','atlas_value'] = i
        mapping.at[f'mrisdp_{302 + i - 3}','atlas_description'] = l_and_r_destrieux[i]
        mapping.at[f'mrisdp_{302 + i - 3}','atlas_value'] = i
        mapping.at[f'mrisdp_{906 + i - 3}','atlas_description'] = l_and_r_destrieux[i]
        mapping.at[f'mrisdp_{906 + i - 3}','atlas_value'] = i

In [15]:
desikan_labels = pd.read_csv(join(PROJ_DIR, DATA_DIR, 'desikan_labels.txt'), 
                             sep='\t', 
                             header=0,
                             usecols=['value']
                            )

In [16]:
cdk_area = mapping.filter(regex='smri_area.*_cdk_.*', axis=0).index
cdk_thick = mapping.filter(regex='smri_thick.*_cdk_.*', axis=0).index
cdk_t1wcnt = mapping.filter(regex='smri_t1wcnt.*_cdk_.*', axis=0).index
cdk_tvar = mapping.filter(regex='rsfmri_var_cdk_.*', axis=0).index

In [17]:
for i in range(0, len(desikan_labels.values)):
    mapping.at[cdk_area[i],'atlas_description'] = desikan_labels.iloc[i]['value']
    mapping.at[cdk_area[i],'atlas_value'] = desikan_labels.index[i]
    mapping.at[cdk_thick[i],'atlas_description'] = desikan_labels.iloc[i]['value']
    mapping.at[cdk_thick[i],'atlas_value'] = desikan_labels.index[i]
    mapping.at[cdk_t1wcnt[i],'atlas_description'] = desikan_labels.iloc[i]['value']
    mapping.at[cdk_t1wcnt[i],'atlas_value'] = desikan_labels.index[i]
    mapping.at[cdk_tvar[i],'atlas_description'] = desikan_labels.iloc[i]['value']
    mapping.at[cdk_tvar[i],'atlas_value'] = desikan_labels.index[i]

In [18]:
scs_1 = ['x',
         'x',
         'x',
         'x',
        "tplh",
        "caudatelh",
        "putamenlh",
        "pallidumlh",
        "bstem",
        "hpuslh",
        "amygdalalh",
        "aal",
        'x',
        'x',
         'x',
        "tprh",
        "caudaterh",
        "putamenrh",
        "pallidumrh",
        "hpusrh",
        "amygdalarh",
        "aar"]

In [19]:
subcort_map = {}
for i in range(0, len(scs_1)):
    if scs_1[i] == 'x':
        pass
    else:
        subcort_map[scs_1[i]] = [subcort.labels[i], i]

In [22]:
all_meas = [i.split('_')[-1] for i in list(mapping.index)]

In [62]:
# assign values to subcortical variables
for key in subcort_map.keys():
    variables = list(mapping.filter(regex=f'.*_scs_{key}.*', axis=0).index)
    for var in variables:
        if not '_cor_' in var:
            #print(var)
            mapping.at[var,'atlas_description'] = subcort_map[key][0]
            mapping.at[var,'atlas_value'] = subcort_map[key][1]

In [19]:
# gordon parcellation
gordon_labels = pd.read_excel(join(PROJ_DIR, 'resources', 'gordon', 'Parcels.xlsx'),
                              header=0, 
                              index_col=0)

/Users/katherine.b/Library/Python/3.8/lib/python/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [20]:
cortgordon = mapping.filter(regex='.*_cortgordon', axis=0).index
for var in cortgordon:
    value = int(var.split('_')[-1][2:-2])
    mapping.at[var, 'atlas_value'] = value
    mapping.at[var, 'atlas_description'] = gordon_labels.loc[value]['Community']

In [ ]:
region_names = {'aal': ('Accumbens', 'L'), 
                'aalh': ('Accumbens', 'L'), 
                'aar': ('Accumbens', 'R'), 
                'aarh': ('Accumbens', 'R'), 
                'ablh': ('Accumbens', 'L'), 
                'abrh' ('Accumbens', 'R'),
                'ad': ('Auditory Network', 'B'), 
                'aglh': ('Amygdala', 'L'), 
                'agrh': ('Amygdala', 'R'), 
                'amygdalalh': ('Amugdala', 'L'), 
                'amygdalarh': ('Amygdala', 'R'), 
                'aomtmlh': (),
                'aomtmrh': (), 
                'atrlh', 
                'atrrh', 
                'banksstslh': ('Banks of Superior Temporal Sulcus', 'L'), 
                'banksstsrh': ('Banks of Superior Temporal Sulcus', 'R'),
                'brainstem': ('Brainstem', 'B'), 
                'bs': ('Brainstem', 'B'), 
                'bstem': ('Brainstem', 'B'), 
                'ca': ('Cinguloparietal Network', 'B'), 
                'caudatelh': ('Caudate', 'L'), 
                'caudaterh': ('Caudate', 'R'),
                'cbclh': ('Cerebellar Cortex', 'L'), 
                'cbcrh': ('Cerebellar Cortex', 'R'), 
                'cbwmlh': ('Cerebellar White Matter', 'L'), 
                'cbwmrh': ('Cerebellar White Matter', 'R'), 
                'cc': ('Corpus Callosum', 'B'), 
                'cdacatelh': ('Anterior Cingulate, Caudal', 'L'),
                'cdacaterh': ('Anterior Cingulate, Caudal', 'R'), 
                'cdaclatelh': ('Anterior Cingulate, Caudal', 'L'), 
                'cdaclaterh': ('Anterior Cingulate, Caudal', 'R'), 
                'cdelh': ('Caudate', 'L'), 
                'cderh': ('Caudate', 'R'), 
                'cdlh': ('Caudate', 'L'),
                'cdmdflh': ('Middle Frontal Gyrus, Caudal', 'L'), 
                'cdmdfrh': ('Middle Frontal Gyrus, Caudal', 'R'), 
                'cdmdfrlh': ('Middle Frontal Gyrus, Caudal', 'L'), 
                'cdmdfrrh': ('Middle Frontal Gyrus, Caudal', 'R'), 
                'cdrh': ('Caudate', 'R'), 
                'cgc': ('Cingulo-Opercular Network', 'B'),
                'cgclh': ('Cingulate Cingulum', 'L'), 
                'cgcrh': ('Cingulate Cingulum', 'R'), 
                'cghlh': ('Parahippocampal Cingulum', 'L'), 
                'cghrh': ('Parahippocampal Cingulum', 'R'),  
                'crbcortexlh': ('Cerebellar Cortex', 'L'),
                'crbcortexrh': ('Cerebellar Cortex', 'R'), 
                'crbwmatterlh': ('Cerebellar White Matter', 'L'), 
                'crbwmatterrh': ('Cerebellar White Matter', 'L'), 
                'crcxlh': ('Cerebellar Cortex', 'L'), 
                'crcxrh': ('Cerebellar Cortex', 'R'),
                'cstlh': ('Corticospinal Tract', 'L'), 
                'cstrh': ('Corticospinal Tract', 'R'), 
                'cuneuslh': ('Cuneus', 'L'), 
                'cuneusrh': ('Cuneus', 'R'), 
                'cwmlh': ('Cerebral White Matter', 'L'), 
                'cwmrh': ('Cerebral White Matter', 'L'), 
                'dla': ('Dorsal Attention Network', 'B'),
                'dlprefrlh': ('Dorsal Prefrontal Cortex', 'L'), 
                'dlprefrrh': ('Dorsal Prefrontal Cortex', 'R'), 
                'dmfrlh': ('Dorsomedial Frontal Cortex', 'L'), 
                'dmfrrh': ('Dorsomedial Frontal Cortex', 'R'), 
                'dt': ('Default Mode Network', 'B'), 
                'ehinallh': ('Entorhinal Cortex', 'L'),
                'ehinalrh': ('Entorhinal Cortex', 'R'), 
                'entorhinallh': ('Entorhinal Cortex', 'L'), 
                'entorhinalrh': ('Entorhinal Cortex', 'R'), 
                'fflh': ('Fusiform Gyrus', 'L'), 
                'ffrh': ('Fusiform Gyrus', 'R'), 
                'fmaj': ('Fornix Major', 'B'),
                'fmin': ('Fornix Minor', 'B'), 
                'fo': ('Frontoparietal Network', 'B'), 
                'fpolelh': ('Frontal Pole', 'L'), 
                'fpolerh': ('Frontal Pole', 'R'), 
                'frpolelh': ('Frontal Pole', 'L'), 
                'frpolerh': ('Frontal Pole', 'R'),
                'fscslh': ('Superior Corticostriate Tract (Frontal)', 'L'), 
                'fscsrh': ('Superior Corticostriate Tract (Frontal)', 'R'), 
                'fusiformlh': ('Fusiform Gyrus', 'L'), 
                'fusiformrh': ('Fusiform Gyrus', 'R'), 
                'fxcutlh': ('Fornix (excluding Fimbria)', 'L'),
                'fxcutrh': ('Fornix (excluding Fimbria)', 'R'), 
                'fxlh': ('Fornix', 'L'), 
                'fxrh': ('Fornix', 'R'), 
                'hclh': ('Hippocampus', 'L'),
                'hcrh': ('Hippocampus', 'R'), 
                'hplh': ('Hippocampus', 'L'), 
                'hprh': ('Hippocampus', 'R'), 
                'hpuslh': ('Hippocampus', 'L'), 
                'hpusrh': ('Hippocampus', 'R'), 
                'ifolh': ('Inferior Fronto-occipital Fasciculus', 'L'), 
                'iforh': ('Inferior Fronto-occipital Fasciculus', 'R'),
                'ifpalh': ('Inferior Parietal', 'L'), 
                'ifparh': ('Inferior Parietal', 'R'), 
                'ifpllh': ('Inferior Parietal', 'L'), 
                'ifplrh': ('Inferior Parietal', 'R'), 
                'ifsfclh', 
                'ifsfcrh',
       'iftlh', 'iftmlh', 'iftmrh', 'iftrh', 'ihcatelh', 'ihcaterh',
       'ihclatelh', 'ihclaterh', 'ilflh', 'ilfrh', 'ilvlh', 'ilvrh',
       'insulalh', 'insularh', 'intracranialv', 'linguallh', 'lingualrh',
       'lobfrlh', 'lobfrrh', 'loboflh', 'lobofrh', 'loccipitallh',
       'loccipitalrh', 'locclh', 'loccrh', 'lvrh', 'mdtlh', 'mdtmlh',
       'mdtmrh', 'mdtrh', 'mobfrlh', 'mobfrrh', 'moboflh', 'mobofrh', 'n',
       'obfrlh', 'obfrrh', 'occlh', 'occrh', 'pallidumlh', 'pallidumrh',
       'paracentrallh', 'paracentralrh', 'paracnlh', 'paracnrh',
       'parahpallh', 'parahpalrh', 'parsobalislh', 'parsobalisrh',
       'parsobislh', 'parsobisrh', 'parsopclh', 'parsopcrh', 'parsopllh',
       'parsoplrh', 'parstgrislh', 'parstgrisrh', 'parstularislh',
       'parstularisrh', 'pclh', 'pcrh', 'pericclh', 'periccrh', 'pllh',
       'plrh', 'postcentrallh', 'postcentralrh', 'postcnlh', 'postcnrh',
       'precentrallh', 'precentralrh', 'precnlh', 'precnrh',
       'precuneuslh', 'precuneusrh', 'psclatelh', 'psclaterh', 'pscslh',
       'pscsrh', 'pslflh', 'pslfrh', 'ptcatelh', 'ptcaterh', 'ptlh',
       'ptoltmlh', 'ptoltmrh', 'ptrh', 'putamenlh', 'putamenrh',
       'rlaclatelh', 'rlaclaterh', 'rlmdflh', 'rlmdfrh', 'rracatelh',
       'rracaterh', 'rrmdfrlh', 'rrmdfrrh', 'rspltp', 'sa', 'scslh',
       'scsrh', 'sifclh', 'sifcrh', 'slflh', 'slfrh', 'smh', 'smlh',
       'smm', 'smrh', 'spetallh', 'spetalrh', 'suflh', 'sufrh', 'sufrlh',
       'sufrrh', 'supllh', 'suplrh', 'sutlh', 'sutmlh', 'sutmrh', 'sutrh',
       'thplh', 'thprh', 'tmpolelh', 'tmpolerh', 'total', 'tplh',
       'tpolelh', 'tpolerh', 'tprh', 'trvtmlh', 'trvtmrh', 'tslflh',
       'tslfrh', 'tvtlh', 'tvtrh', 'unclh', 'uncrh', 'vdclh', 'vdcrh',
       'vedclh', 'vedcrh', 'ventraldclh', 'ventraldcrh', 'vs', 'vta',
       'vtdclh', 'vtdcrh'}

In [33]:
# skipping fuzzy12 because the atlases are only available upon email request
# and I doubt I'll use them for plotting anyway
# next up: fiber atlas
FIBER_DIR = '/Volumes/projects_herting/LABDOCS/PROJECTS/ABCD/ABCD_Outcomes/ABCD-brain outcomes/AtlasTrack/'
atlastrack_rois = join(FIBER_DIR, 'fibertract_visualization', 'ABCD Atlas Track ROIs')
fiber_key = pd.read_csv(join(FIBER_DIR, 'documentation', 'DTI_Fiber_Legend.csv'), 
                        header=0, 
                        index_col=0)

In [72]:
!ls /Volumes/projects_herting/LABDOCS/PROJECTS/ABCD/ABCD_Outcomes/ABCD-brain\ outcomes/AtlasTrack/fibertract_visualization/ABCD\ Atlas\ Track\ Tracts

Cingulum_L_yellow.tt
Cingulum_L_yellow.tt.gz
Cingulum_Parahippocampal_L_yellow.tt.gz
Cingulum_Parahippocampal_R.tt.gz
Cingulum_Parahippocampal_R_yellow.tt.gz
Cingulum_R_yellow.tt.gz
Corpus_Callosum_Body_red.tt.gz
Corpus_Callosum_Forceps_Major_red.tt.gz
Corpus_Callosum_Forceps_Minor_red.tt.gz
Corpus_Callosum_Tapetum_red.tt.gz
Corticospinal_Tract_L_blue.tt.gz
Corticospinal_Tract_R.tt.gz
Corticospinal_Tract_R_blue.tt.gz
Fornix_L.tt.gz
Fornix_L_magenta.tt.gz
Fornix_R.tt.gz
Fornix_R_magenta.tt.gz
Inferior_Fronto_Occipital_Fasciculus_L_yellowgreen.tt.gz
Inferior_Fronto_Occipital_Fasciculus_R_yellowgreen.tt.gz
Inferior_Longitudinal_Fasciculus_L_purple.tt.gz
Inferior_Longitudinal_Fasciculus_R_purple.tt.gz
Superior_Longitudinal_Fasciculus3_L_cyan.tt.gz
Superior_Longitudinal_Fasciculus3_R.tt.gz
Superior_Longitudinal_Fasciculus3_R_cyan.tt.gz
Thalamic_Radiation_Anterior_L.tt.gz
Thalamic_Radiation_Anterior_L_green.tt.gz
Thalamic_Radiation_Anterior_L_right.tt.gz
Thalamic_Radiation_Anterior_R_green.t

In [34]:
fiber_key['FiberName'] = [name.lower() for name in fiber_key['FiberName']]

In [66]:
fiber_key

,FiberName,LongFiberName,ReorderNumber,ColorRGB,ColorDescription
FiberNumber,,,,,
101,r_fx,R fornix,8,128 32 64,magneta
102,l_fx,L fornix,9,128 32 64,magneta
103,r_cgc,R cingulum (cingulate),4,232 232 0,bright yellow
104,l_cgc,L cingulum (cingulate),5,232 232 0,bright yellow
105,r_cgh,R cingulum (parahippocampal),6,232 232 0,bright yellow
106,l_cgh,L cingulum (parahippocampal),7,232 232 0,bright yellow
107,r_cst,R corticospinal tract or pyramidal tract,16,0 0 255,bright blue
108,l_cst,L corticospinal tract or pyramidal tract,17,0 0 255,bright blue
109,r_atr,R anterior thalamic radiations,24,0 255 0,bright green


In [49]:
for var in mapping.filter(regex='.*_fiber', axis=0).index:
    tract_name = var.split('_')[-1]
    hemisphere = tract_name[-2]
    if not 'h' in var:
        tract = tract_name
        index = fiber_key[fiber_key['FiberName'] == tract].index
    else:
        tract = tract_name[:-2]
        index = fiber_key[fiber_key['FiberName'] == f'{hemisphere}_{tract}'].index
        mapping.at[var, 'atlas_description'] = fiber_key.loc[index]['LongFiberName']
        mapping.at[var, 'atlas_value'] = int(index.values)
        mapping.at[var, 'atlas_fname'] = join(atlastrack_rois, f'fiber_{int(index.values)}_count_countatlas.nii.gz')

In [50]:
for var in mapping.filter(regex='rsi.*_fib.*', axis=0).index:
    tract_name = var.split('_')[-1]
    hemisphere = tract_name[-2]
    if not 'h' in var:
        tract = tract_name
        index = fiber_key[fiber_key['FiberName'] == tract].index
    else:
        tract = tract_name[:-2]
        index = fiber_key[fiber_key['FiberName'] == f'{hemisphere}_{tract}'].index
        mapping.at[var, 'atlas_description'] = fiber_key.loc[index]['LongFiberName']
        mapping.at[var, 'atlas_value'] = int(index.values)
        mapping.at[var, 'atlas_fname'] = join(atlastrack_rois, f'fiber_{int(index.values)}_count_countatlas.nii.gz')

In [75]:
corrs = mapping.filter(regex='rsfmri_cor_ngd.*', axis=0).index
corrs = [i.split('.')[0] for i in corrs]
subcort = list(np.unique([i.split('_')[-1] for i in corrs]))

corrs = mapping.filter(regex='rsfmri_c_ngd.*', axis=0).index
corrs = [i.split('.')[0] for i in corrs]
networks = list(np.unique([i.split('_')[-1] for i in corrs]))

In [76]:
print(subcort)

['aalh', 'aarh', 'aglh', 'agrh', 'bs', 'cdelh', 'cderh', 'crcxlh', 'crcxrh', 'hplh', 'hprh', 'pllh', 'plrh', 'ptlh', 'ptrh', 'thplh', 'thprh', 'vtdclh', 'vtdcrh']


In [63]:
mapping.to_csv(join(PROJ_DIR, DATA_DIR, 'variable_to_nifti_mapping.csv'))

In [ ]:
destrieux_vol_nimg = nib.load(destrieux_vol.maps)
destrieux_vol_arra = destrieux_vol_nimg.get_fdata()

In [86]:
desikan_vol_nimg = nib.load(desikan.images[0])
desikan_vol_arra = desikan_vol_nimg.get_fdata()